In [1]:
# Parameters
nPC = 50
batchKeys = "chemistry"
queryDataset = "EXTERNAL"


In [2]:
for v in ['nPC','batchKeys', 'queryDataset']:
    if v in locals() or v in globals():
        print(f"{v} = {eval(v)}")
    else:
        raise Exception(f"{v} not specified")

nPC = 50
batchKeys = chemistry
queryDataset = EXTERNAL


In [3]:
import os
import sys

import scanpy as sc
import pandas as pd
import numpy as np

import symphonypy as sp

from sklearn.metrics import balanced_accuracy_score

from pyprojroot import here

sys.path.insert(1, str(here('bin')))
# Import custom functions
from customPythonFunctions import aggregating_features, train_patient_classifier, vote_patient_disease

random_seed = 42

### Parameters

In [4]:
batch_key_query = batchKeys.split(',')
batch_key_query

['chemistry']

### Loading dataset

**Integrated object**

In [5]:
adataR = sc.read_h5ad(here(f"03_downstream_analysis/08_PatientClassifier/Harmony_Symphony/results/01_MAIN_HarmonyIntegrated_{nPC}nPC_Batch_{batchKeys}.h5ad"), backed='r')
adataR

AnnData object with n_obs × n_vars = 4435922 × 8253 backed at '/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/03_downstream_analysis/08_PatientClassifier/Harmony_Symphony/results/01_MAIN_HarmonyIntegrated_50nPC_Batch_chemistry.h5ad'
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'disease', 'sex', 'binned_age', 'Level1', 'Level2'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status', 'highly_variable', 'mean', 'std'
    uns: 'harmony', 'log1p', 'pca'
    obsm: 'X_pca', 'X_pca_harmony'
    varm: 'PCs'

**Query dataset**

In [6]:
adataQ = sc.read_h5ad(here(f'03_downstream_analysis/02_gene_universe_definition/results/05_{queryDataset}_geneUniverse.h5ad'))
adataQ

AnnData object with n_obs × n_vars = 572872 × 8253
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'disease', 'sex', 'binned_age'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status'

### Preprocessing

In [7]:
sc.pp.normalize_total(adataQ, target_sum=1e4)
sc.pp.log1p(adataQ)

### Projection with Symphony

In [8]:
sp.tl.map_embedding(adataQ, adataR, key=batch_key_query)

In [9]:
adataQ

AnnData object with n_obs × n_vars = 572872 × 8253
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'disease', 'sex', 'binned_age'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status'
    uns: 'log1p'
    obsm: 'X_pca_reference', 'X_pca_harmony', 'X_pca_harmony_symphony_R'

### Label transfer

In [10]:
# common embedded space -> adataQ.obsm['X_pca_harmony']

**Level1 Annotation**

In [11]:
sp.tl.transfer_labels_kNN(adataQ, adataR, 'Level1', ref_basis = 'X_pca_harmony', query_basis = 'X_pca_harmony')

**Disease**

In [12]:
adataQ.obs.rename({'disease':'disease_true'}, axis=1, inplace=True)

In [13]:
sp.tl.transfer_labels_kNN(adataQ, adataR, 'disease', ref_basis = 'X_pca_harmony', query_basis = 'X_pca_harmony')

In [14]:
adataQ.obs.rename({'disease':'disease_pred'}, axis=1, inplace=True)

In [15]:
balanced_accuracy_score(y_true = adataQ.obs.disease_true, y_pred=adataQ.obs.disease_pred)

/scratch_isilon/groups/singlecell/shared/conda_env/inflammation_atlas_R1/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


0.16127949434823827

In [16]:
adataQ.obs

,studyID,libraryID,sampleID,chemistry,technology,disease_true,sex,binned_age,Level1,disease_pred
cellID,,,,,,,,,,
SCGT00val_L006_I036016_T0_AAACCCAAGACAACTA,SCGT00val,SCGT00val_L006,SCGT00val_I036016_T0,3_GEX_V3,3_GEX_V3_GenoHashed,RA,female,31-40,T_CD4_Naive,HIV
SCGT00val_L006_I036023_T0_AAACCCAAGACGGAAA,SCGT00val,SCGT00val_L006,SCGT00val_I036023_T0,3_GEX_V3,3_GEX_V3_GenoHashed,RA,female,51-60,T_CD4_NonNaive,RA
SCGT00val_L006_I036016_T0_AAACCCAAGAGCATTA,SCGT00val,SCGT00val_L006,SCGT00val_I036016_T0,3_GEX_V3,3_GEX_V3_GenoHashed,RA,female,31-40,T_CD4_Naive,HIV
SCGT00val_L006_I036021_T0_AAACCCAAGATGGTCG,SCGT00val,SCGT00val_L006,SCGT00val_I036021_T0,3_GEX_V3,3_GEX_V3_GenoHashed,RA,female,61-70,T_CD8_NonNaive,SLE
SCGT00val_L006_I036021_T0_AAACCCAAGCAACAAT,SCGT00val,SCGT00val_L006,SCGT00val_I036021_T0,3_GEX_V3,3_GEX_V3_GenoHashed,RA,female,61-70,DC,UC
...,...,...,...,...,...,...,...,...,...,...
10XGenomics_L008_10XHC8_T0_TTTGTCATCCACGTTC,10XGenomics,10XGenomics_L008,10XGenomics_10XHC8_T0,5_GEX_V2,5_GEX_V2,healthy,na,NaN,B,healthy
10XGenomics_L008_10XHC8_T0_TTTGTCATCCCGACTT,10XGenomics,10XGenomics_L008,10XGenomics_10XHC8_T0,5_GEX_V2,5_GEX_V2,healthy,na,NaN,T_CD4_Naive,healthy
10XGenomics_L008_10XHC8_T0_TTTGTCATCGTGGGAA,10XGenomics,10XGenomics_L008,10XGenomics_10XHC8_T0,5_GEX_V2,5_GEX_V2,healthy,na,NaN,T_CD4_Naive,healthy


### Generating pseudobulk from latent space for Q dataset

In [17]:
emb_pseudobulk_Q = aggregating_features(Z = adataQ.obsm['X_pca_harmony'], obsDF = adataQ.obs[['sampleID','Level1','disease_true']], mode = 'mean', obs_names_col=['sampleID','Level1'])

In [18]:
emb_pseudobulk_Q.obs

,sampleID,Level1,disease_true,n_observation
10XGenomics_10XHC1_T0_B,10XGenomics_10XHC1_T0,B,healthy,979
10XGenomics_10XHC1_T0_Cycling_cells,10XGenomics_10XHC1_T0,Cycling_cells,healthy,21
10XGenomics_10XHC1_T0_DC,10XGenomics_10XHC1_T0,DC,healthy,127
10XGenomics_10XHC1_T0_ILC,10XGenomics_10XHC1_T0,ILC,healthy,315
10XGenomics_10XHC1_T0_Mono,10XGenomics_10XHC1_T0,Mono,healthy,1463
...,...,...,...,...
Savage2021_PIDB_T0_T_CD4_NonNaive,Savage2021_PIDB_T0,T_CD4_NonNaive,healthy,1752
Savage2021_PIDB_T0_T_CD8_Naive,Savage2021_PIDB_T0,T_CD8_Naive,healthy,370
Savage2021_PIDB_T0_T_CD8_NonNaive,Savage2021_PIDB_T0,T_CD8_NonNaive,healthy,2430
Savage2021_PIDB_T0_UTC,Savage2021_PIDB_T0,UTC,healthy,178


**Loading training pseudobulk**

In [19]:
emb_pseudobulk_train = sc.read_h5ad(here(f"03_downstream_analysis/08_PatientClassifier/Harmony_Symphony/results/01_MAIN_HarmonyIntegrated_PSEUDOBULK_{nPC}nPC_Batch_{batchKeys}.h5ad"))

#### Remove unwanted cell types

In [20]:
emb_pseudobulk_train = emb_pseudobulk_train[~emb_pseudobulk_train.obs.Level1.isin(['Cycling_cells','RBC','Progenitors','Platelets']),]

In [21]:
emb_pseudobulk_Q = emb_pseudobulk_Q[~emb_pseudobulk_Q.obs.Level1.isin(['Cycling_cells','RBC','Progenitors','Platelets']),]

### Saving query dataset and pseudobulks

In [22]:
adataQ.write(here(f"03_downstream_analysis/08_PatientClassifier/Harmony_Symphony/results/02_{queryDataset}_SymphonyProjected_{nPC}nPC_Batch_{batchKeys}.h5ad"), 
             compression='gzip')

In [23]:
emb_pseudobulk_Q.write(here(f"03_downstream_analysis/08_PatientClassifier/Harmony_Symphony/results/02_{queryDataset}_SymphonyProjected_PSEUDOBULK{nPC}nPC_Batch_{batchKeys}.h5ad"), 
             compression='gzip')

/scratch_isilon/groups/singlecell/shared/conda_env/inflammation_atlas_R1/lib/python3.10/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/scratch_isilon/groups/singlecell/shared/conda_env/inflammation_atlas_R1/lib/python3.10/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/scratch_isilon/groups/singlecell/shared/conda_env/inflammation_atlas_R1/lib/python3.10/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


### Training linear classifiers

In [24]:
clfList = train_patient_classifier(adataTrain = emb_pseudobulk_train, cell_type_col = 'Level1', y_true_col = 'disease', max_iter=-1, random_state = 25, model = 'KNeighborsClassifier', 
                           kargs_model={'n_neighbors':2, 'weights':'uniform', 'metric':'cosine','n_jobs':-1})

  0%|          | 0/11 [00:00<?, ?it/s]

 18%|█▊        | 2/11 [00:00<00:01,  7.87it/s]

 27%|██▋       | 3/11 [00:00<00:01,  7.67it/s]

 36%|███▋      | 4/11 [00:00<00:01,  4.24it/s]

 45%|████▌     | 5/11 [00:00<00:01,  4.84it/s]

...

 73%|███████▎  | 8/11 [00:01<00:00,  6.04it/s]

 82%|████████▏ | 9/11 [00:01<00:00,  6.51it/s]

 91%|█████████ | 10/11 [00:01<00:00,  6.49it/s]

100%|██████████| 11/11 [00:01<00:00,  6.63it/s]

100%|██████████| 11/11 [00:01<00:00,  6.03it/s]

In [25]:
clfList

{'B': {'clf': KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=2),
  'bAcc': 0.8855346334951598,
  'nObs': 814},
 'DC': {'clf': KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=2),
  'bAcc': 0.8798902229551049,
  'nObs': 801},
 'ILC': {'clf': KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=2),
  'bAcc': 0.8601168149657772,
  'nObs': 817},
 'Mono': {'clf': KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=2),
  'bAcc': 0.8796345484540444,
  'nObs': 817},
 'Plasma': {'clf': KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=2),
  'bAcc': 0.7968588090216773,
  'nObs': 727},
 'T_CD4_Naive': {'clf': KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=2),
  'bAcc': 0.8950121248147564,
  'nObs': 814},
 'T_CD4_NonNaive': {'clf': KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=2),
  'bAcc': 0.913420880516387,
  'nObs': 816},
 'T_CD8_Naive': {'clf': KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=2),
  '

### Predicting diseases

In [26]:
emb_pseudobulk_Q.obs

,sampleID,Level1,disease_true,n_observation
10XGenomics_10XHC1_T0_B,10XGenomics_10XHC1_T0,B,healthy,979
10XGenomics_10XHC1_T0_DC,10XGenomics_10XHC1_T0,DC,healthy,127
10XGenomics_10XHC1_T0_ILC,10XGenomics_10XHC1_T0,ILC,healthy,315
10XGenomics_10XHC1_T0_Mono,10XGenomics_10XHC1_T0,Mono,healthy,1463
10XGenomics_10XHC1_T0_Plasma,10XGenomics_10XHC1_T0,Plasma,healthy,4
...,...,...,...,...
Savage2021_PIDB_T0_T_CD4_NonNaive,Savage2021_PIDB_T0,T_CD4_NonNaive,healthy,1752
Savage2021_PIDB_T0_T_CD8_Naive,Savage2021_PIDB_T0,T_CD8_Naive,healthy,370
Savage2021_PIDB_T0_T_CD8_NonNaive,Savage2021_PIDB_T0,T_CD8_NonNaive,healthy,2430
Savage2021_PIDB_T0_UTC,Savage2021_PIDB_T0,UTC,healthy,178


In [27]:
classificationDF = vote_patient_disease(adataTest = emb_pseudobulk_Q, 
                                clfList = clfList, 
                                sample_id_col = 'sampleID',
                                cell_type_col = 'Level1')


  0%|          | 0/11 [00:00<?, ?it/s]

 45%|████▌     | 5/11 [00:00<00:00, 42.85it/s]

 91%|█████████ | 10/11 [00:00<00:00, 42.39it/s]

100%|██████████| 11/11 [00:00<00:00, 42.39it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

100%|██████████| 86/86 [00:00<00:00, 966.87it/s]

In [28]:
res = classificationDF.merge(emb_pseudobulk_Q.obs[['sampleID', 'disease_true']].drop_duplicates(), how = 'left', on = 'sampleID').set_index('sampleID')


In [29]:
balanced_accuracy_score(y_true = res.disease_true, y_pred=res.firstChoice)

/scratch_isilon/groups/singlecell/shared/conda_env/inflammation_atlas_R1/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


0.21622242647058823

In [30]:
res

,B_prediction,DC_prediction,ILC_prediction,Mono_prediction,Plasma_prediction,T_CD4_Naive_prediction,T_CD4_NonNaive_prediction,T_CD8_Naive_prediction,T_CD8_NonNaive_prediction,UTC_prediction,pDC_prediction,firstChoice,firstChoice_perc,secondChoice,secondChoice_perc,disease_true
sampleID,,,,,,,,,,,,,,,,
10XGenomics_10XHC1_T0,NPC,healthy,healthy,SLE,NPC,NPC,NPC,NPC,healthy,healthy,healthy,NPC,38.461538,healthy,38.461538,healthy
10XGenomics_10XHC2_T0,NPC,healthy,healthy,healthy,COVID,NPC,NPC,NPC,healthy,healthy,healthy,healthy,46.153846,NPC,30.769231,healthy
10XGenomics_10XHC3_T0,SLE,healthy,healthy,SLE,SLE,PS,healthy,healthy,healthy,healthy,healthy,healthy,53.846154,SLE,23.076923,healthy
10XGenomics_10XHC4_T0,SLE,healthy,COPD,SLE,SLE,healthy,healthy,healthy,NPC,healthy,healthy,healthy,46.153846,SLE,23.076923,healthy
10XGenomics_10XHC5_T0,SLE,healthy,healthy,SLE,SLE,healthy,healthy,healthy,healthy,healthy,healthy,healthy,61.538462,SLE,23.076923,healthy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Savage2021_BRISL5_T0,SLE,SLE,HNSCC,COVID,HNSCC,COVID,COVID,NPC,COVID,HNSCC,healthy,COVID,30.769231,HNSCC,23.076923,SLE
Savage2021_BRISL6_T0,HNSCC,healthy,HNSCC,healthy,HNSCC,healthy,healthy,healthy,healthy,HNSCC,healthy,healthy,53.846154,HNSCC,30.769231,SLE
Savage2021_BRISL7_T0,SLE,healthy,COVID,HNSCC,COVID,NPC,NPC,NPC,COVID,COVID,healthy,COVID,30.769231,NPC,23.076923,SLE
